# 特徵工程 (Feature Engineering)
負責算 MA, RSI, MACD (Feature Engineering)

In [4]:
import pandas as pd
import numpy as np

def calculate_rsi(series, period=14):
    """
    輔助函式：計算 RSI (相對強弱指標)
    """
    delta = series.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()

    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

def calculate_macd(series, fast=12, slow=26, signal=9):
    """
    輔助函式：計算 MACD
    回傳: DIF, DEM, OSC
    """
    ema_fast = series.ewm(span=fast, adjust=False).mean()
    ema_slow = series.ewm(span=slow, adjust=False).mean()
    dif = ema_fast - ema_slow
    dem = dif.ewm(span=signal, adjust=False).mean()
    osc = dif - dem
    return dif, dem, osc

def add_technical_indicators(df):
    """
    主函式：為每一支股票加上 MA, RSI, MACD 特徵
    """
    df = df.copy()
    
    # 1. 找出所有收盤價欄位
    close_cols = [c for c in df.columns if 'close' in c.lower()]
    
    print(f"📊 開始特徵工程 (MA, RSI, MACD)，共處理 {len(close_cols)} 支股票...")

    for col in close_cols:
        prefix = col.split('_')[0] 
        
        # --- A. MA ---
        df[f'{prefix}_MA_5'] = df[col].rolling(window=5).mean()
        df[f'{prefix}_MA_10'] = df[col].rolling(window=10).mean()
        
        # --- B. RSI ---
        df[f'{prefix}_RSI_14'] = calculate_rsi(df[col], period=14)
        
        # --- C. MACD ---
        dif, dem, osc = calculate_macd(df[col])
        df[f'{prefix}_MACD_DIF'] = dif
        df[f'{prefix}_MACD_DEM'] = dem
        df[f'{prefix}_MACD_OSC'] = osc
        
        # --- D. 乖離率 ---
        df[f'{prefix}_Bias_5'] = (df[col] - df[f'{prefix}_MA_5']) / df[f'{prefix}_MA_5']

        # # --- D. 收益率 (Returns) - 最重要！ ---
        # df[f'{prefix}_Return_1'] = df[col].pct_change(periods=1)
        
        # # --- E. 波動率 (Volatility/Std) ---
        # # 過去 20 天的標準差，代表風險大小
        # df[f'{prefix}_Std_20'] = df[col].rolling(window=20).std()
        
        # # --- F. 布林通道位置 (Bollinger Band Position) ---
        # ma_20 = df[col].rolling(window=20).mean()
        # std_20 = df[col].rolling(window=20).std()
        # upper = ma_20 + (2 * std_20)
        # lower = ma_20 - (2 * std_20)
        # # 避免分母為 0
        # denominator = (upper - lower)
        # # 如果上下軌距離太近(趨近0)，就給 0.5 (中間值)，不然會報錯
        # df[f'{prefix}_BB_Pos'] = np.where(denominator == 0, 0.5, (df[col] - lower) / denominator)

    # 2. 補值
    df = df.bfill().fillna(0)
    
    print(f"✅ 特徵工程完成！總欄位數: {len(df.columns)}")
    return df

In [5]:
# --- 這是測試區塊 ---
if __name__ == "__main__":
    import pandas as pd
    
    # 1. 隨便造一點假資料來測試，或是讀取真實檔案
    # 這裡我們造一個簡單的 DataFrame
    print("🧪 正在進行自我測試...")
    df_test = pd.DataFrame({
        'date': range(20),
        '1101_close': [30 + i + (i%3) for i in range(20)] # 假裝股價在漲
    })
    
    # 2. 呼叫您寫好的函式！(這時候才會真的跑 print)
    df_result = add_technical_indicators(df_test)
    
    # 3. 印出結果看看
    print("測試結果預覽：")
    print(df_result.tail())

🧪 正在進行自我測試...
📊 開始特徵工程 (MA, RSI, MACD)，共處理 1 支股票...
✅ 特徵工程完成！總欄位數: 9
測試結果預覽：
    date  1101_close  1101_MA_5  1101_MA_10  1101_RSI_14  1101_MACD_DIF  \
15    15          45       44.0        41.4    78.260870       3.705041   
16    16          47       44.8        42.5    78.260870       3.924215   
17    17          49       46.2        43.6    83.333333       4.210757   
18    18          48       47.0        44.4    78.260870       4.307498   
19    19          50       47.8        45.5    78.260870       4.493747   

    1101_MACD_DEM  1101_MACD_OSC  1101_Bias_5  
15       2.781730       0.923311     0.022727  
16       3.010227       0.913988     0.049107  
17       3.250333       0.960424     0.060606  
18       3.461766       0.845732     0.021277  
19       3.668162       0.825585     0.046025  
